# 2. 파이썬으로 데이터 주무르기, pandas
**pandas를 활용해서 데이터프레임을 다뤄봅시다.**

1. Pandas 시작하기
    - prerequisite : Table
    - pandas import하기
   
2. Pandas로 1차원 데이터 다루기 - Series 
    - Series 선언하기
    - Series vs ndarray
    - Series vs dict
    - Series에 이름 붙이기
3. Pandas로 2차원 데이터 다루기 - dataframe
    - dataframe 선언하기
    - from csv to dataframe
    - dataframe 자료 접근하기

[수업에 사용된 covid 데이터](https://www.kaggle.com/imdevskp/corona-virus-report)

## I. pandas 시작하기

## II. pandas로 1차원 데이터 다루기 - Series

## III. Pandas로 2차원 데이터 다루기 - dataframe

## Mission:
### 1. covid 데이터에서 100 case 대비 사망률(`Deaths / 100 Cases`)이 가장 높은 국가는?

In [3]:
import pandas as pd
covid = pd.read_csv("./country_wise_latest.csv")
#hi = covid["Deaths / 100 Cases"] == max(covid["Deaths / 100 Cases"])
#hi
country = covid[covid["Deaths / 100 Cases"] == max(covid["Deaths / 100 Cases"])]["Country/Region"]
country

184    Yemen
Name: Country/Region, dtype: object

In [37]:
covid

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean
183,Western Sahara,10,1,8,1,0,0,0,10.00,80.00,12.50,10,0,0.00,Africa
184,Yemen,1691,483,833,375,10,4,36,28.56,49.26,57.98,1619,72,4.45,Eastern Mediterranean
185,Zambia,4552,140,2815,1597,71,1,465,3.08,61.84,4.97,3326,1226,36.86,Africa


### 2. covid 데이터에서 신규 확진자가 없는 나라 중 WHO Region이 'Europe'를 모두 출력하면?  
Hint : 한 줄에 동시에 두가지 조건을 Apply하는 경우 Warning이 발생할 수 있습니다.

In [85]:
covid_no_new_case = covid[covid["New cases"]==0]
# covid_no_new_case
covid_no_new_europe = covid_no_new_case[covid_no_new_case["WHO Region"] == "Europe"]
# covid_no_new_europe
covid_no_new_europe["Country/Region"]

56           Estonia
75          Holy See
95            Latvia
100    Liechtenstein
113           Monaco
143       San Marino
157            Spain
Name: Country/Region, dtype: object

### 3. 다음 [데이터](https://www.kaggle.com/neuromusic/avocado-prices)를 이용해 각 Region별로 아보카도가 가장 비싼 평균가격(AveragePrice)을 출력하면?

In [7]:
avocado = pd.read_csv("../avocado.csv")
# avocado
avocado_by_region = avocado["AveragePrice"].groupby(by=avocado["region"])

mean_AveragePrice = avocado_by_region.mean().sort_values(axis=0, ascending=False)
mean_AveragePrice.head(1)

region
HartfordSpringfield    1.818639
Name: AveragePrice, dtype: float64